In [1]:
!python --version

Python 3.10.12


In [2]:
!git clone https://github.com/decodingml/personalized-recommender-course.git
%cd personalized-recommender-course

Cloning into 'personalized-recommender-course'...
remote: Enumerating objects: 927, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 927 (delta 41), reused 50 (delta 27), pack-reused 837 (from 1)
Receiving objects: 100% (927/927), 4.38 MiB | 19.02 MiB/s, done.
Resolving deltas: 100% (565/565), done.
/content/personalized-recommender-course


In [3]:
!make install

Streaming output truncated to the last 5000 lines.
nvidia-cublas-cu12 ------------------------------ 191.67 MiB/346.60 MiB
tensorflow ------------------------------ 195.74 MiB/467.18 MiB
nvidia-cudnn-cu12 ------------------------------ 191.95 MiB/633.96 MiB
⠇ Preparing packages... (191/200)
plotly     ------------------------------ 11.05 MiB/14.12 MiB
catboost   ------------------------------ 58.75 MiB/93.98 MiB
nvidia-cusparse-cu12 ------------------------------ 191.44 MiB/197.84 MiB
nvidia-cufft-cu12 ------------------------------ 190.99 MiB/201.66 MiB
triton     ------------------------------ 188.15 MiB/241.43 MiB
nvidia-cublas-cu12 ------------------------------ 191.91 MiB/346.60 MiB
tensorflow ------------------------------ 195.74 MiB/467.18 MiB
nvidia-cudnn-cu12 ------------------------------ 191.95 MiB/633.96 MiB
⠇ Preparing packages... (191/200)
plotly     ------------------------------ 11.07 MiB/14.12 MiB
catboost   ------------------------------ 58.75 MiB/93.98 MiB
nvidia-cus

In [4]:
!uv run python --version
# Output: Python 3.11.8

⠙ Preparing packages... (0/72)
⠙ Preparing packages... (0/72)
⠙ Preparing packages... (0/72)
click      ------------------------------     0 B/95.65 KiB
⠙ Preparing packages... (0/72)
click      ------------------------------     0 B/95.65 KiB
pyparsing  ------------------------------ 14.88 KiB/104.42 KiB
⠙ Preparing packages... (0/72)
click      ------------------------------     0 B/95.65 KiB
pyparsing  ------------------------------ 14.88 KiB/104.42 KiB
jinja2     ------------------------------     0 B/130.15 KiB
⠙ Preparing packages... (0/72)
click      ------------------------------     0 B/95.65 KiB
pyparsing  ------------------------------ 14.88 KiB/104.42 KiB
jinja2     ------------------------------     0 B/130.15 KiB
prompt-toolkit ------------------------------     0 B/377.53 KiB
⠙ Preparing packages... (0/72)
click      ------------------------------ 14.93 KiB/95.65 KiB
pyparsing  ------------------------------ 14.88 KiB/104.42 KiB
jinja2     ------------------------------ 

In [5]:
!cp .env.example .env

In [6]:
!make all

uv run ipython notebooks/1_fp_computing_features.ipynb
]0;IPython: content/personalized-recommender-courseNo event loop hook running.
Traceback (most recent call last):
  File "/content/personalized-recommender-course/.venv/bin/ipython", line 8, in <module>
    sys.exit(start_ipython())
             ^^^^^^^^^^^^^^^
  File "/content/personalized-recommender-course/.venv/lib/python3.11/site-packages/IPython/__init__.py", line 130, in start_ipython
    return launch_new_instance(argv=argv, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/personalized-recommender-course/.venv/lib/python3.11/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/content/personalized-recommender-course/.venv/lib/python3.11/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/personalized-recommender-course/.venv

In [7]:
import time

notebook_start_time = time.time()

# Set up environment

In [9]:
import sys
from pathlib import Path


def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False


def clone_repository() -> None:
    !git clone https://github.com/decodingml/hands-on-recommender-system.git
    %cd hands-on-recommender-system/


def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml


if is_google_colab():
    clone_repository()
    install_dependencies()

    root_dir = str(Path().absolute())
    print("⛳️ Google Colab environment")
else:
    root_dir = str(Path().absolute().parent)
    print("⛳️ Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    print(f"Adding the following directory to the PYTHONPATH: {root_dir}")
    sys.path.append(root_dir)

Cloning into 'hands-on-recommender-system'...
remote: Enumerating objects: 927, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 927 (delta 41), reused 50 (delta 27), pack-reused 837 (from 1)
Receiving objects: 100% (927/927), 4.38 MiB | 14.03 MiB/s, done.
Resolving deltas: 100% (565/565), done.
/content/personalized-recommender-course/hands-on-recommender-system/hands-on-recommender-system
Using Python 3.10.12 environment at: /usr
Resolved 187 packages in 104ms
Audited 187 packages in 0.20ms
⛳️ Google Colab environment
Adding the following directory to the PYTHONPATH: /content/personalized-recommender-course/hands-on-recommender-system/hands-on-recommender-system


# 👩🏻‍🔬 Feature pipeline: Computing features

## Imports

In [10]:
%load_ext autoreload
%autoreload 2

import warnings
from pprint import pprint

import polars as pl
import torch
from loguru import logger
from sentence_transformers import SentenceTransformer

warnings.filterwarnings("ignore")

from recsys import hopsworks_integration
from recsys.config import settings
from recsys.features.articles import (
    compute_features_articles,
    generate_embeddings_for_dataframe,
)
from recsys.features.customers import DatasetSampler, compute_features_customers
from recsys.features.interaction import generate_interaction_data
from recsys.features.ranking import compute_ranking_dataset
from recsys.features.transactions import compute_features_transactions
from recsys.hopsworks_integration import feature_store
from recsys.raw_data_sources import h_and_m as h_and_m_raw_data

## Constants

These are the default settings used across the lessons. You can always override them in the `.env` file that sits at the root of the repository:

In [11]:
pprint(dict(settings))

{'CUSTOMER_DATA_SIZE': <CustomerDatasetSize.SMALL: 'SMALL'>,
 'CUSTOM_HOPSWORKS_INFERENCE_ENV': 'custom_env_name',
 'FEATURES_EMBEDDING_MODEL_ID': 'all-MiniLM-L6-v2',
 'HOPSWORKS_API_KEY': None,
 'OPENAI_API_KEY': None,
 'OPENAI_MODEL_ID': 'gpt-4o-mini',
 'RANKING_DATASET_VALIDATON_SPLIT_SIZE': 0.1,
 'RANKING_EARLY_STOPPING_ROUNDS': 5,
 'RANKING_ITERATIONS': 100,
 'RANKING_LEARNING_RATE': 0.2,
 'RANKING_MODEL_TYPE': 'ranking',
 'RANKING_SCALE_POS_WEIGHT': 10,
 'RECSYS_DIR': PosixPath('/content/personalized-recommender-course/hands-on-recommender-system/hands-on-recommender-system/recsys'),
 'TWO_TOWER_DATASET_TEST_SPLIT_SIZE': 0.1,
 'TWO_TOWER_DATASET_VALIDATON_SPLIT_SIZE': 0.1,
 'TWO_TOWER_LEARNING_RATE': 0.01,
 'TWO_TOWER_MODEL_BATCH_SIZE': 2048,
 'TWO_TOWER_MODEL_EMBEDDING_SIZE': 16,
 'TWO_TOWER_NUM_EPOCHS': 10,
 'TWO_TOWER_WEIGHT_DECAY': 0.001}


The most important one is the dataset size.

Choosing a different dataset size will impact the time it takes to run everything and the quality of the final models. We suggest using a small dataset size when running this the first time.

Suported user dataset sizes:

In [12]:
DatasetSampler.get_supported_sizes()

{<CustomerDatasetSize.LARGE: 'LARGE'>: 50000,
 <CustomerDatasetSize.MEDIUM: 'MEDIUM'>: 5000,
 <CustomerDatasetSize.SMALL: 'SMALL'>: 1000}

## <span style="color:#ff5f27">🔮 Connect to Hopsworks Feature Store </span>

In [13]:
project, fs = hopsworks_integration.get_feature_store()

2025-02-11 11:29:06.931 | INFO     | recsys.hopsworks_integration.feature_store:get_feature_store:18 - Login to Hopsworks using cached API key.


Copy your Api Key (first register/login): https://c.app.hopsworks.ai/account/api/generated

Paste it here: ··········

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213603


# The H&M dataset

To show how a recommender system using the two tower architecture works, we will use the [H&M Personalized Fashion Recommendations](https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations) dataset.

It consists of:
- articles
- customers
- transactions

# 🗄️ Articles data

The **article_id** and **product_code** serve different purposes in the context of H&M's product database:

- **Article ID**: This is a unique identifier assigned to each individual article within the database. It is typically used for internal tracking and management purposes. Each distinct item or variant of a product (e.g., different sizes or colors) would have its own unique article_id.

- **Product Code**: This is also a unique identifier, but it is associated with a specific product or style rather than individual articles. It represents a broader category or type of product within H&M's inventory. Multiple articles may share the same product code if they belong to the same product line or style.

While both are unique identifiers, the article_id is specific to individual items, whereas the product_code represents a broader category or style of product.

Here is an example:

**Product: Basic T-Shirt**

- **Product Code:** TS001

- **Article IDs:**
    - Article ID: 1001 (Size: Small, Color: White)
    - Article ID: 1002 (Size: Medium, Color: White)
    - Article ID: 1003 (Size: Large, Color: White)
    - Article ID: 1004 (Size: Small, Color: Black)
    - Article ID: 1005 (Size: Medium, Color: Black)

In this example, "TS001" is the product code for the basic t-shirt style. Each variant of this t-shirt (e.g., different sizes and colors) has its own unique article_id.



In [14]:
articles_df = h_and_m_raw_data.extract_articles_df()
articles_df.shape

(105542, 25)

The articles DataFrame looks as follows:

In [15]:
articles_df.head(3)

article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,perceived_colour_master_id,perceived_colour_master_name,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
i64,i64,str,i64,str,str,i64,str,i64,str,i64,str,i64,str,i64,str,str,str,i64,str,i64,str,i64,str,str
108775015,108775,"""Strap top""",253,"""Vest top""","""Garment Upper body""",1010016,"""Solid""",9,"""Black""",4,"""Dark""",5,"""Black""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""","""Jersey top with narrow shoulde…"
108775044,108775,"""Strap top""",253,"""Vest top""","""Garment Upper body""",1010016,"""Solid""",10,"""White""",3,"""Light""",9,"""White""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""","""Jersey top with narrow shoulde…"
108775051,108775,"""Strap top (1)""",253,"""Vest top""","""Garment Upper body""",1010017,"""Stripe""",11,"""Off White""",1,"""Dusty Light""",9,"""White""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""","""Jersey top with narrow shoulde…"


Check for NaNs:


In [16]:
articles_df.null_count()

article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,perceived_colour_master_id,perceived_colour_master_name,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,416


## Articles feature engineering


In [17]:
articles_df = compute_features_articles(articles_df)
articles_df.shape


(105542, 27)

The features of the articles look as:

In [18]:
articles_df.head(3)

article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,perceived_colour_master_id,perceived_colour_master_name,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,prod_name_length,article_description,image_url
str,i64,str,i64,str,str,i64,str,i64,str,i64,str,i64,str,i64,str,str,str,i64,str,i64,str,i64,str,u32,str,str
"""108775015""",108775,"""Strap top""",253,"""Vest top""","""Garment Upper body""",1010016,"""Solid""",9,"""Black""",4,"""Dark""",5,"""Black""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""",9,"""Strap top - Vest top in Garmen…","""https://repo.hops.works/dev/jd…"
"""108775044""",108775,"""Strap top""",253,"""Vest top""","""Garment Upper body""",1010016,"""Solid""",10,"""White""",3,"""Light""",9,"""White""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""",9,"""Strap top - Vest top in Garmen…","""https://repo.hops.works/dev/jd…"
"""108775051""",108775,"""Strap top (1)""",253,"""Vest top""","""Garment Upper body""",1010017,"""Stripe""",11,"""Off White""",1,"""Dusty Light""",9,"""White""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""",13,"""Strap top (1) - Vest top in Ga…","""https://repo.hops.works/dev/jd…"


### Create embeddings from the articles description

In [19]:
for i, desc in enumerate(articles_df["article_description"].head(n=3)):
    logger.info(f"Item {i+1}:\n{desc}")

2025-02-11 11:29:46.946 | INFO     | __main__:<cell line: 1>:2 - Item 1:
Strap top - Vest top in Garment Upper body
Appearance: Solid
Color: Dark Black (Black)
Category: Ladieswear - Womens Everyday Basics - Jersey Basic
Details: Jersey top with narrow shoulder straps.
2025-02-11 11:29:46.948 | INFO     | __main__:<cell line: 1>:2 - Item 2:
Strap top - Vest top in Garment Upper body
Appearance: Solid
Color: Light White (White)
Category: Ladieswear - Womens Everyday Basics - Jersey Basic
Details: Jersey top with narrow shoulder straps.
2025-02-11 11:29:46.950 | INFO     | __main__:<cell line: 1>:2 - Item 3:
Strap top (1) - Vest top in Garment Upper body
Appearance: Stripe
Color: Dusty Light White (Off White)
Category: Ladieswear - Womens Everyday Basics - Jersey Basic
Details: Jersey top with narrow shoulder straps.


In [27]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
logger.info(
    f"Loading '{settings.FEATURES_EMBEDDING_MODEL_ID}' embedding model to {device=}"
)
print(f"{device=}")

# Load the embedding model from SentenceTransformer's model registry.
model = SentenceTransformer(settings.FEATURES_EMBEDDING_MODEL_ID, device=device)
model.to('cuda')
print(f"f{model.device=}")

2025-02-11 11:50:11.320 | INFO     | __main__:<cell line: 8>:8 - Loading 'all-MiniLM-L6-v2' embedding model to device='cuda'


device='cuda'
fmodel.device=device(type='cuda', index=0)


In [29]:
articles_df = generate_embeddings_for_dataframe(
    articles_df, "article_description", model, batch_size=10024
)  # Reduce batch size if getting OOM errors.

Generating embeddings:   0%|          | 0/105542 [00:00<?, ?it/s]

For each article description, we have a numerical vector which we can feed to a model, opposite to a string containing the description of an object.

In [30]:
articles_df[["article_description", "embeddings"]].head(3)

article_description,embeddings
str,list[f64]
"""Strap top - Vest top in Garmen…","[-0.026782, 0.082344, … 0.022782]"
"""Strap top - Vest top in Garmen…","[-0.010396, 0.089874, … 0.022564]"
"""Strap top (1) - Vest top in Ga…","[-0.032753, 0.091124, … 0.022804]"


### Looking at image links

In [31]:
articles_df["image_url"][0]

'https://repo.hops.works/dev/jdowling/h-and-m/images/010/0108775015.jpg'

In [32]:
from IPython.display import HTML, display

image_urls = articles_df["image_url"].tail(12).to_list()
grid_html = '<div style="display: grid; grid-template-columns: repeat(6, 1fr); gap: 10px; max-width: 900px;">'

for url in image_urls:
    grid_html += f'<img src="{url}" style="width: 100%; height: auto;">'

grid_html += "</div>"

display(HTML(grid_html))


## Customers Data

In [33]:
customers_df = h_and_m_raw_data.extract_customers_df()
customers_df.shape


(1371980, 7)

The customers DataFrame looks as follows:

In [34]:
customers_df.head(3)

customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
str,f64,f64,str,str,i64,str
"""00000dbacae5abe5e23885899a1fa4…",null,null,"""ACTIVE""","""NONE""",49,"""52043ee2162cf5aa7ee79974281641…"
"""0000423b00ade91418cceaf3b26c6a…",null,null,"""ACTIVE""","""NONE""",25,"""2973abc54daa8a5f8ccfe9362140c6…"
"""000058a12d5b43e67d225668fa1f8d…",null,null,"""ACTIVE""","""NONE""",24,"""64f17e6a330a85798e4998f62d0930…"


Check for NaNs:

In [35]:

customers_df.null_count()

customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
u32,u32,u32,u32,u32,u32,u32
0,895050,907576,6062,16009,15861,0


## Customers feature engineering


In [36]:
customers_df = compute_features_customers(customers_df, drop_null_age=True)
customers_df.shape

(1356119, 5)

The features of the customers DataFrame looks as follows:

In [37]:
customers_df.head(3)

customer_id,club_member_status,age,postal_code,age_group
str,str,f64,str,str
"""00000dbacae5abe5e23885899a1fa4…","""ACTIVE""",49.0,"""52043ee2162cf5aa7ee79974281641…","""46-55"""
"""0000423b00ade91418cceaf3b26c6a…","""ACTIVE""",25.0,"""2973abc54daa8a5f8ccfe9362140c6…","""19-25"""
"""000058a12d5b43e67d225668fa1f8d…","""ACTIVE""",24.0,"""64f17e6a330a85798e4998f62d0930…","""19-25"""



# Transactions Data

In [38]:
transactions_df = h_and_m_raw_data.extract_transactions_df()
transactions_df.shape

(31788324, 5)

The transaction DataFrame looks as follows:

In [39]:
transactions_df.head(3)

t_dat,customer_id,article_id,price,sales_channel_id
date,str,i64,f64,i64
2018-09-20,"""000058a12d5b43e67d225668fa1f8d…",663713001,0.050831,2
2018-09-20,"""000058a12d5b43e67d225668fa1f8d…",541518023,0.030492,2
2018-09-20,"""00007d2de826758b65a93dd24ce629…",505221004,0.015237,2


## Transactions feature engineering

In [40]:
transactions_df = compute_features_transactions(transactions_df)
transactions_df.shape

(31788324, 9)

The time of the year a purchase was made should be a strong predictor, as seasonality plays a big factor in fashion purchases. Here, you will use the month of the purchase as a feature. Since this is a cyclical feature (January is as close to December as it is to February), you'll map each month to the unit circle using sine and cosine.

Thus, the features of the transactions DataFrame look as follows:

In [41]:
transactions_df.head(3)

t_dat,customer_id,article_id,price,sales_channel_id,year,month,day,day_of_week
i64,str,str,f64,i64,i32,i8,i8,i8
0,"""000058a12d5b43e67d225668fa1f8d…","""663713001""",0.050831,2,2018,9,20,4
0,"""000058a12d5b43e67d225668fa1f8d…","""541518023""",0.030492,2,2018,9,20,4
0,"""00007d2de826758b65a93dd24ce629…","""505221004""",0.015237,2,2018,9,20,4


We don't want to work with ~30 million transactions in these series, as everything will take too much time to run. Thus, we create a subset of the original dataset by randomly sampling from the customers' datasets and taking only their transactions.

In [42]:
sampler = DatasetSampler(size=settings.CUSTOMER_DATA_SIZE)
dataset_subset = sampler.sample(
    customers_df=customers_df, transations_df=transactions_df
)
customers_df = dataset_subset["customers"]
transactions_df = dataset_subset["transactions"]

2025-02-11 11:55:35.997 | INFO     | recsys.features.customers:sample:29 - Sampling 1000 customers.
2025-02-11 11:55:36.136 | INFO     | recsys.features.customers:sample:32 - Number of transactions for all the customers: 31788324
2025-02-11 11:55:39.177 | INFO     | recsys.features.customers:sample:38 - Number of transactions for the 1000 sampled customers: 23799


In [43]:
transactions_df.shape

(23799, 9)

Some of the remaining customers:

In [44]:
for customer_id in transactions_df["customer_id"].unique().head(10):
    logger.info(f"Logging customer ID: {customer_id}")

2025-02-11 11:55:39.911 | INFO     | __main__:<cell line: 1>:2 - Logging customer ID: d9448c8585f1678937deb5118d95b09bf6f41fe00a65b1fb82c7d176c6bfc532
2025-02-11 11:55:39.913 | INFO     | __main__:<cell line: 1>:2 - Logging customer ID: e3dcf6b3aeed395753967e3d78c4557ddcfdd2941c247e41f7663142a351b1e2
2025-02-11 11:55:39.915 | INFO     | __main__:<cell line: 1>:2 - Logging customer ID: c0a482a27be291f7b3b54f0f3ebd8098dae7628431d2a8a2df316ec75c34139d
2025-02-11 11:55:39.917 | INFO     | __main__:<cell line: 1>:2 - Logging customer ID: 0b2c678f22ff13943e6a9637b1d2b9b26f3688ddcd455e5d18564fa6ce24ffd2
2025-02-11 11:55:39.921 | INFO     | __main__:<cell line: 1>:2 - Logging customer ID: 5bd4f47e25726c917387ab1a5c9b91435792a3807c0f66acb82380624e688302
2025-02-11 11:55:39.925 | INFO     | __main__:<cell line: 1>:2 - Logging customer ID: 6d450227f6bff204f4237e88850e25e7cbeeeda2294a81bdd604a68c9d821050
2025-02-11 11:55:39.933 | INFO     | __main__:<cell line: 1>:2 - Logging customer ID: 00b203a3

# 🤳🏻 Interaction data

To train our models, we need more than just the transactions DataFrame. We need positive samples that signal whether a customer clicked or bought an item, but we also need negative samples that signal no interactions between a customer and an item.

In [45]:
interaction_df = generate_interaction_data(transactions_df)
interaction_df.shape

Processing customer chunks: 100%|██████████| 1/1 [00:08<00:00,  8.61s/it]


(135918, 5)

The interaction features look as follows:

In [46]:
interaction_df.head()

t_dat,customer_id,article_id,interaction_score,prev_article_id
i64,str,str,i64,str
-381600000,"""00b203a32faa3d007dba198ef27c15…","""849868002""",0,"""START"""
-370800000,"""00b203a32faa3d007dba198ef27c15…","""881916003""",0,"""849868002"""
-370800000,"""00b203a32faa3d007dba198ef27c15…","""881916003""",0,"""881916003"""
-363600000,"""00b203a32faa3d007dba198ef27c15…","""754323003""",0,"""881916003"""
-360000000,"""00b203a32faa3d007dba198ef27c15…","""753360002""",0,"""754323003"""


Let's take a look at the interaction score distribution:

In [47]:
interaction_df.group_by("interaction_score").agg(
    pl.count("interaction_score").alias("total_interactions")
)

interaction_score,total_interactions
i64,u32
0,73801
1,38318
2,23799


Here is what each score means:
- `0` : No interaction between a customer and an item
- `1` : A customer clicked an item
- `2` : A customer bought an item

# <span style="color:#ff5f27">🪄 Create Hopsworks Feature Groups </span>

A [feature group](https://docs.hopsworks.ai/feature-store-api/latest/generated/feature_group/) can be seen as a collection of conceptually related features.

To create a feature group you need to give it a name and specify a primary key. It is also best practice to provide a description of the contents of the feature group.

#### Customers

We set `online_enabled=True` to enable low-latency access to the data from the inference pipeline for real-time predictions.

A full list of arguments can be found in the [documentation](https://docs.hopsworks.ai/feature-store-api/latest/generated/api/feature_store_api/#create_feature_group).

In [48]:
logger.info("Uploading 'customers' Feature Group to Hopsworks.")
customers_fg = feature_store.create_customers_feature_group(
    fs, df=customers_df, online_enabled=True
)

logger.info("✅ Uploaded 'customers' Feature Group to Hopsworks!")

2025-02-11 11:55:49.512 | INFO     | __main__:<cell line: 1>:1 - Uploading 'customers' Feature Group to Hopsworks.


Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1213603/fs/1201233/fg/1400633


Uploading Dataframe: 100.00% |██████████| Rows 1000/1000 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: customers_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1213603/jobs/named/customers_1_offline_fg_materialization/executions


2025-02-11 11:58:28.742 | INFO     | __main__:<cell line: 6>:6 - ✅ Uploaded 'customers' Feature Group to Hopsworks!


#### Articles

Let's do the same thing for the rest of the data frames.

In [49]:
logger.info("Uploading 'articles' Feature Group to Hopsworks.")
articles_fg = feature_store.create_articles_feature_group(
    fs,
    df=articles_df,
    articles_description_embedding_dim=model.get_sentence_embedding_dimension(),
    online_enabled=True,
)
logger.info("✅ Uploaded 'articles' Feature Group to Hopsworks!")


2025-02-11 11:58:28.834 | INFO     | __main__:<cell line: 1>:1 - Uploading 'articles' Feature Group to Hopsworks.


Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1213603/fs/1201233/fg/1400634


Uploading Dataframe: 100.00% |██████████| Rows 105542/105542 | Elapsed Time: 01:30 | Remaining Time: 00:00


Launching job: articles_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1213603/jobs/named/articles_1_offline_fg_materialization/executions


2025-02-11 12:04:23.575 | INFO     | __main__:<cell line: 8>:8 - ✅ Uploaded 'articles' Feature Group to Hopsworks!


####  Transactions

In [50]:
logger.info("Uploading 'transactions' Feature Group to Hopsworks.")
trans_fg = feature_store.create_transactions_feature_group(
    fs=fs, df=transactions_df, online_enabled=True
)
logger.info("✅ Uploaded 'transactions' Feature Group to Hopsworks!")

2025-02-11 12:04:23.712 | INFO     | __main__:<cell line: 1>:1 - Uploading 'transactions' Feature Group to Hopsworks.


Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1213603/fs/1201233/fg/1398850


Uploading Dataframe: 100.00% |██████████| Rows 23799/23799 | Elapsed Time: 00:03 | Remaining Time: 00:00


Launching job: transactions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1213603/jobs/named/transactions_1_offline_fg_materialization/executions


2025-02-11 12:07:25.920 | INFO     | __main__:<cell line: 5>:5 - ✅ Uploaded 'transactions' Feature Group to Hopsworks!


#### Interactions

In [51]:
logger.info("Uploading 'interactions' Feature Group to Hopsworks.")
interactions_fg = feature_store.create_interactions_feature_group(
    fs=fs, df=interaction_df, online_enabled=True
)
logger.info("✅ Uploaded 'interactions' Feature Group to Hopsworks!!")

2025-02-11 12:07:26.057 | INFO     | __main__:<cell line: 1>:1 - Uploading 'interactions' Feature Group to Hopsworks.


Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1213603/fs/1201233/fg/1398851


Uploading Dataframe: 100.00% |██████████| Rows 135918/135918 | Elapsed Time: 00:05 | Remaining Time: 00:00


Launching job: interactions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1213603/jobs/named/interactions_1_offline_fg_materialization/executions


2025-02-11 12:10:08.762 | INFO     | __main__:<cell line: 5>:5 - ✅ Uploaded 'interactions' Feature Group to Hopsworks!!


# Compute ranking dataset

The last step is to compute the ranking dataset used to train the scoring/ranking model from the feature groups we've just created:


In [52]:
ranking_df = compute_ranking_dataset(
    trans_fg,
    articles_fg,
    customers_fg,
)
ranking_df.shape

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.68s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (7.68s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.90s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (31.87s) 


(224136, 15)

The ranking dataset looks as follows:

In [53]:
ranking_df.head(3)

customer_id,age,article_id,label,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,perceived_colour_value_name,perceived_colour_master_name,department_name,index_name,index_group_name,section_name,garment_group_name
str,f64,str,i32,str,str,str,str,str,str,str,str,str,str,str
"""34d30dcece38ac652e9fd05285d94d…",64.0,"""839496003""",1,"""Top""","""Garment Upper body""","""All over pattern""","""Beige""","""Dark""","""Beige""","""Jersey fancy""","""Ladieswear""","""Ladieswear""","""Womens Everyday Collection""","""Jersey Fancy"""
"""91a30617b9a5b5ff3927779204a176…",49.0,"""777093001""",1,"""Jumpsuit/Playsuit""","""Garment Full body""","""All over pattern""","""Dark Blue""","""Dark""","""Blue""","""Young Girl Dresses""","""Children Sizes 134-170""","""Baby/Children""","""Young Girl""","""Dresses/Skirts girls"""
"""23eeb5e9595c9409031f21a9c01fa3…",25.0,"""875719003""",1,"""Trousers""","""Garment Lower body""","""Solid""","""Beige""","""Dusty Light""","""Mole""","""Trousers DS""","""Divided""","""Divided""","""Divided Selected""","""Trousers"""


In [54]:
ranking_df.get_column("label").value_counts()

label,count
i32,u32
1,20376
0,203760


As the ranking dataset was computed based on articles, customers, and transactions Hopsworks Feature Groups, we can reflect this lineage in the ranking Feature Group.

In [55]:
logger.info("Uploading 'ranking' Feature Group to Hopsworks.")
rank_fg = feature_store.create_ranking_feature_group(
    fs,
    df=ranking_df,
    parents=[articles_fg, customers_fg, trans_fg],
    online_enabled=False
)
logger.info("✅ Uploaded 'ranking' Feature Group to Hopsworks!!")

2025-02-11 12:10:58.152 | INFO     | __main__:<cell line: 1>:1 - Uploading 'ranking' Feature Group to Hopsworks.


Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1213603/fs/1201233/fg/1400635


Uploading Dataframe: 100.00% |██████████| Rows 224136/224136 | Elapsed Time: 00:15 | Remaining Time: 00:00


Launching job: ranking_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1213603/jobs/named/ranking_1_offline_fg_materialization/executions


2025-02-11 12:14:34.914 | INFO     | __main__:<cell line: 8>:8 - ✅ Uploaded 'ranking' Feature Group to Hopsworks!!


## <span style="color:#ff5f27"> Inspecting the Feature Groups in Hopsworks UI </span>

View results in [Hopsworks Serverless](https://rebrand.ly/serverless-github): **Feature Store → Feature Groups**

---

In [56]:
notebook_end_time = time.time()
notebook_execution_time = notebook_end_time - notebook_start_time

logger.info(
    f"⌛️ Notebook Execution time: {notebook_execution_time:.2f} seconds ~ {notebook_execution_time / 60:.2f} minutes"
)

2025-02-11 12:14:35.005 | INFO     | __main__:<cell line: 4>:4 - ⌛️ Notebook Execution time: 2821.76 seconds ~ 47.03 minutes


# <span style="color:#ff5f27">→ Next Steps </span>

In the next notebook you'll train the retrieval model and register it to the Hopsworks model registry.